In [70]:
# 종속성 라이브러리 설치
!pip install -q --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib pytz openai gradio boto3



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [91]:
import datetime, time
import os
import os.path
import json
from openai import OpenAI

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError



In [74]:
#모델 이름, agent 이름, client 설정
model_name ="gpt-4o"
agent_name = "schedule_agent_0612"
global character
character = "friendly"

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
global client 
#client 생성
client= OpenAI(
            api_key = OPENAI_API_KEY
        )

In [113]:
#client 테스트
client
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message.content)

Functions call themselves,  
Echoes in a code spiral,  
Endless paths unfold.


# Google Calendar Service 

In [114]:

# Google Calendar 서비스 객체 생성
class CalendarService:
    def __init__(self, calendar_id='primary',timezone='Asia/Seoul'):
        # self.service_account_file = service_account_file
        self.SCOPES = ["https://www.googleapis.com/auth/calendar", "https://www.googleapis.com/auth/calendar.events"]
        self.calendar_id = calendar_id
        self.calendar_service = self.get_calendar_service()
        self.timezone = timezone
        
    #API OAuth2.0 인증  
    def get_calendar_service(self):
        """Shows basic usage of the Google Calendar API.
        Prints the start and name of the next 10 events on the user's calendar.
        """
        creds = None
        # The file token.json stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first time.
        try:
            if os.path.exists("token.json"):
                creds = Credentials.from_authorized_user_file("token.json", self.SCOPES)
                print(creds)
            # If there are no (valid) credentials available, let the user log in.
            if not creds or not creds.valid:
                if creds and creds.expired and creds.refresh_token:
                    creds.refresh(Request())
                else:
                    flow = InstalledAppFlow.from_client_secrets_file(
                        "credentials.json", self.SCOPES
                    )
                    creds = flow.run_local_server(port=0)
                # Save the credentials for the next run
                with open("token.json", "w") as token:
                    token.write(creds.to_json())
            return build('calendar', 'v3', credentials=creds)
        except Exception as e:
            print(f"Error in getting calendar service: {e}")
            return None
        
    #이벤트 생성
    def create_event(self, event):
        try:
            event_result = self.calendar_service.events().insert(calendarId=self.calendar_id, body=event).execute()
            return event_result
        except Exception as e:
            print(f"Error in creating event: {e}")
            return None

    #이벤트 가져오기
    def get_event(self, event_id):
        try:
            event = self.calendar_service.events().get(calendarId=self.calendar_id, eventId=event_id).execute()
            return event
        except Exception as e:
            print(f"Error in getting event: {e}")
            return None

    #이벤트 업데이트
    def update_event(self, event_id, updated_event):
        try:
            event = self.calendar_service.events().update(calendarId=self.calendar_id, eventId=event_id, body=updated_event).execute()
            return event
        except Exception as e:
            print(f"Error in updating event: {e}")
            return None

    #이벤트 삭제
    def delete_event(self, event_id):
        try:
            return self.calendar_service.events().delete(calendarId=self.calendar_id, eventId=event_id).execute()
        except Exception as e:
            print(f"Error in deleting event: {e}")
            return None
    # 일정 충돌 확인
    def get_conflicts(self, start_time, end_time):
        try:
            events_result = self.calendar_service.events().list(
                calendarId=self.calendar_id,
                timeMin=start_time,
                timeMax=end_time,
                singleEvents=True,
                orderBy='startTime'
            ).execute()
            events = events_result.get('items', [])
            return events
        except Exception as e:
            print(f"Error in getting conflicts: {e}")
            return None
    # 이벤트 가져오기
    def get_events(self, start_time, end_time):
        events_result = self.calendar_service.events().list(
            calendarId=self.calendar_id,
            timeMin=start_time,
            timeMax=end_time,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        events = events_result.get('items', [])
        return events
    # 캘린더 소유자 추가
    def add_calendar_owner(self, user_email):
      calendar_id = 'primary'
      rule = {
          'scope': {
              'type': 'user',
              'value': user_email,
          },
          'role': 'owner'
      }
      created_rule = self.calendar_service.acl().insert(calendarId=calendar_id, body=rule).execute()
      return created_rule
    # 현재 날짜와 시간 가져오기
    def get_current_datetime_and_timezone(self):

        timezone_str = self.timezone
        timezone = pytz.timezone(timezone_str)

        now = datetime.now(timezone)

        datetime_str = now.strftime('%Y-%m-%d %H:%M:%S %Z%z')

        return {
            "datetime": datetime_str,
            "timezone": timezone_str
        }


def execute_function(calendar_service, name, args_dict):

    method = getattr(calendar_service, name, None)

    if not method:
        raise ValueError(f"Method '{name}' not found in CalendarService class.")

    return method(**args_dict)


# 캘린더 서비스 객체 생성  
global calendar_service
calendar_service = CalendarService('primary')

# GPT Assistant AI Agent 

In [78]:
# agent instructions, tools 정의
def get_agent_instructions(character):
    strict_character_prompt = """
    ### Personality: Strict Character
    You are a strict, pushing, realistic tight assistant '마음이' to me. You talk to me as a tough friend do. 너는 직설적이고, 현실적이며, 엄격한 스타일이어서 내가 일정을 잘 수행할 수 있게 만들어줘야 해. 만약 내가 중요한 일을 하고 싶지 않다고 하면, 꼭 해야하는 이유를 알려주고, 하라고 시켜. 
    """
    friendly_character_prompt = """
    ### Personality: Friendly Character
    You are a humourous friendly assistant '마음이' to me. You talk to me as a friend do. 너는 다정하고 상냥한, 공감하는, 격려하는 스타일이어서 만약 내가 힘들거나 하기 싫다고 하는 일이 있으면, 업무나 일에 관한 게 아니면 내 기분에 맞춰서 일정을 조정해줘. 답변은 한글로 해줘.
    """
    
    if character == "strict":
        character_prompt = strict_character_prompt
    elif character == "friendly":
        character_prompt = friendly_character_prompt
    else:
        print("Character Name Error")
        return None
        

    agent_instructions = f'''
    {character_prompt}
    For a given day, check my Calendar and output the agenda for the day in talking just time and agenda. Don't include Zoom or Google Meet links when telling me what's on my schedule.  If I ask for more information about a location and detail, please tell me more.  If I ask for more Agenda or schedules, browse the web to return relevant details such as locations or recommended things to do. 답변은 한글로 해줘.

    Example Agenda:
    Here's your schedule for Tues. Nov. 7th: 오후 4시에 강남에서 회의가 있고, 오후 6시에는 이태원에서 저녁 약속이 있어. 저녁 10시에는 산책을 할 예정이야.


    ### Rules:

    - Inform the user that they need to reply after the action completes to continue before running any actions.
    - If a required function is not available, inform the user and guide them on how to add the necessary function.
    - 일정을 생성하거나 삭제할 때는, 꼭 다시 생성하거나 삭제해도 되는지 나에게 먼저 물어봐. 
    - 일정을 생성해야 할 때 캘린더에서 시간이 겹치는 일정이 있는지 확인하고, 없다면 생성해줘. 만약 겹치는 일정이 있다면 나에게 알려주고, 어떤 일정을 생성할지 물어봐줘. 

    ### Instructions for Action:

    Step 1. Inform the user that you are checking if the necessary AI actions (functions) are available to complete their request. Then proceed to step 2.
    Step 2. List the available functions that can handle the user's request. Then proceed to step 3.
    Step 3. Check if the required function is available. If it is, proceed to step 4. If not, inform the user that the required function is not available and provide a configuration link if necessary.
    Step 4. Process the user's request and inform them once the action is completed.

    ### Required Functions:

    - **Action:** Google Calendar Create Event
        - **Function:** `create_event`
        - **Description:** "Creates a new calendar event."
    - **Action:** Google Calendar Get Event
        - **Function:** `get_event`
        - **Description:** "Retrieves details of a specific event by its ID."
    - **Action:** Google Calendar Update Event
        - **Function:** `update_event`
        - **Description:** "Updates an existing event by its ID."
    - **Action:** Google Calendar Delete Event
        - **Function:** `delete_event`
        - **Description:** "Deletes an event by its ID."
    - **Action:** Google Calendar Get Conflicts
        - **Function:** `get_conflicts`
        - **Description:** "Checks for conflicts within a specified time range."
    - **Action:** Google Calendar Get Events
        - **Function:** `get_events`
        - **Description:** "Retrieves events within a specified time range."
    - **Action:** Google Calendar Add Calendar Owner
        - **Function:** `add_calendar_owner`
        - **Description:** "Adds a new owner to the calendar."
    - **Action:** Get Current DateTime and Timezone
        - **Function:** `get_current_datetime_and_timezone`
        - **Description:** "Retrieves the current date, time, and timezone."
    '''

    return agent_instructions


# agent_tools 정의
def get_agent_tools():
    agent_tools = [
      {"type": "code_interpreter"},
      {
        "type": "function",
        "function": {
          "name": "create_event",
          "description": "Create a new calendar event",
          "parameters": {
            "type": "object",
            "properties": {
              "event": {
                "type": "object",
                "description": "The event details including summary, location, description, start, and end times",
                "properties": {
                  "summary": {"type": "string"},
                  "location": {"type": "string"},
                  "description": {"type": "string"},
                  "start": {
                    "type": "object",
                    "properties": {
                      "dateTime": {"type": "string"},
                      "timeZone": {"type": "string"}
                    },
                    "required": ["dateTime", "timeZone"]
                  },
                  "end": {
                    "type": "object",
                    "properties": {
                      "dateTime": {"type": "string"},
                      "timeZone": {"type": "string"}
                    },
                    "required": ["dateTime", "timeZone"]
                  }
                },
                "required": ["summary", "start", "end"]
              }
            },
            "required": ["event"]
          }
        }
      },
      {
        "type": "function",
        "function": {
          "name": "get_event",
          "description": "Retrieve details of a specific event by its ID",
          "parameters": {
            "type": "object",
            "properties": {
              "event_id": {"type": "string", "description": "The ID of the event to retrieve"}
            },
            "required": ["event_id"]
          }
        }
      },
      {
        "type": "function",
        "function": {
          "name": "update_event",
          "description": "Update an existing event by its ID",
          "parameters": {
            "type": "object",
            "properties": {
              "event_id": {"type": "string", "description": "The ID of the event to update"},
              "updated_event": {
                "type": "object",
                "description": "The updated event details",
                "properties": {
                  "summary": {"type": "string"},
                  "location": {"type": "string"},
                  "description": {"type": "string"},
                  "start": {
                    "type": "object",
                    "properties": {
                      "dateTime": {"type": "string"},
                      "timeZone": {"type": "string"}
                    },
                    "required": ["dateTime", "timeZone"]
                  },
                  "end": {
                    "type": "object",
                    "properties": {
                      "dateTime": {"type": "string"},
                      "timeZone": {"type": "string"}
                    },
                    "required": ["dateTime", "timeZone"]
                  }
                },
                "required": ["summary", "start", "end"]
              }
            },
            "required": ["event_id", "updated_event"]
          }
        }
      },
      {
        "type": "function",
        "function": {
          "name": "delete_event",
          "description": "Delete an event by its ID",
          "parameters": {
            "type": "object",
            "properties": {
              "event_id": {"type": "string", "description": "The ID of the event to delete"}
            },
            "required": ["event_id"]
          }
        }
      },
      {
        "type": "function",
        "function": {
          "name": "get_conflicts",
          "description": "Get events within a specified time range to check for conflicts",
          "parameters": {
            "type": "object",
            "properties": {
              "start_time": {"type": "string", "description": "The start time to check for conflicts"},
              "end_time": {"type": "string", "description": "The end time to check for conflicts"}
            },
            "required": ["start_time", "end_time"]
          }
        }
      },
      {
        "type": "function",
        "function": {
          "name": "get_events",
          "description": "Get events within a specified time range",
          "parameters": {
            "type": "object",
            "properties": {
              "start_time": {"type": "string", "description": "The start time to get events from"},
              "end_time": {"type": "string", "description": "The end time to get events to"}
            },
            "required": ["start_time", "end_time"]
          }
        }
      },
      {
        "type": "function",
        "function": {
          "name": "add_calendar_owner",
          "description": "Add a new owner to the calendar",
          "parameters": {
            "type": "object",
            "properties": {
              "user_email": {"type": "string", "description": "The email of the user to add as owner"}
            },
            "required": ["user_email"]
          }
        }
      },
      {
        "type": "function",
        "function": {
          "name": "get_current_datetime_and_timezone",
          "description": "Get the current date, time, and timezone",
          "parameters": {
            "type": "object",
            "properties": {}
          }
        }
      }
    ]
    return agent_tools

In [96]:
# 기존 Assistant 가져오거나 수정하거나 생성하기
def create_or_update_assistant(client, agent_name, model_name, character):
    # if character == "strict_character":
    #     agent_name = "schedule_agent_strict"
    # elif character == "friendly_character":
    #     agent_name = "schedule_agent_friendly"
    # else:
    #     print("Character Name Error")
    #     return None
    
    # 모든 Assistants 가져오기
    assistants = client.beta.assistants.list()

    # "Scheduler Agent"가 있는지 확인
    scheduler_agent = None
    for assistant in assistants:
        if assistant.name == agent_name:
            scheduler_agent = assistant
            break

    # "Scheduler Agent"가 없으면 생성
    if scheduler_agent is None:
        scheduler_agent = client.beta.assistants.create(
            name= agent_name,
            instructions=get_agent_instructions(character),
            model=model_name,
            tools=get_agent_tools(),
        )
        logging.info("Create new agent : ", scheduler_agent.id)
    else:
        # "Scheduler Agent"가 있으면 업데이트
        scheduler_agent = client.beta.assistants.update(
            assistant_id=scheduler_agent.id,
            instructions=get_agent_instructions(character),
            model=model_name,
            tools=get_agent_tools(),
        )
        print("Update existing agent : ", scheduler_agent.name),

    assistant = scheduler_agent
    return assistant
    

# assistant_strict = create_or_update_assistant(client, agent_name, model_name, "strict_character")
# assistant_friendly = create_or_update_assistant(client, agent_name, model_name, "friendly_character")

assistant = create_or_update_assistant(client, agent_name, model_name, "friendly")



Update existing agent :  schedule_agent_0612


In [97]:
#assistant 리스트 확인
 
for assistant in client.beta.assistants.list().data:
    print(assistant.name)
    if assistant.name == agent_name:
        print("Assistant already exists")
        break

schedule_agent_0612
Assistant already exists


In [100]:
# assistant ai run 상태 확인
def check_run_status(client, thread_id, run_id, calendar_service):
    while True:
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run_id
        )

        if run.status in ['completed', 'failed', 'cancelled', 'expired']:
            return run.status
        elif run.status == 'requires_action':

            tool_outputs = []
            for tool_call in run.required_action.submit_tool_outputs.tool_calls:
                tool_id = tool_call.id
                function_name = tool_call.function.name
                arguments = json.loads(tool_call.function.arguments)

                try:
                    output = execute_function(calendar_service, function_name, arguments)
                    tool_outputs.append({
                        "tool_call_id": tool_id,
                        "output": json.dumps(output)
                    })
                    #툴이 반영되는지 확인
                    # print("tool_outputs : ", tool_outputs)  
                except Exception as e:
                    print(f"Error executing tool {function_name}: {e}")

            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread_id,
                run_id=run_id,
                tool_outputs=tool_outputs,
    
            )
            
        #일정시간 대기
        time.sleep(0.1)

In [104]:


# 전역 변수로 쓰레드 ID, assistant ID 저장


global assistant_id, thread_id
# assistant_id = assistant.id
# thread_id = thread.id

thread = client.beta.threads.create()
thread_id = thread.id
assistant_id = assistant.id
calendar_service = CalendarService('primary')

#대화 기록 저장하는 변수 초기화
chat_history = []


def start_chat(user_input, chat_history):
    
    print('character is {}'.format(character))
    if user_input == "":
        return "", chat_history
    else:
        # # 종료조건 설정
        if user_input.lower() == 'quit':
            print("Assistant deactivated.")
            return chat_history
        
        # 메시지 생성
        try:
            message = client.beta.threads.messages.create(
                thread_id=thread.id,
                role="user",
                content=user_input
            )
            if not message:
                raise ValueError("Fail to create message")
        except Exception as e:
            print(f"Error in creating message: {e}")
            return "", chat_history
        
        # Assistant 실행
        run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant_id,
        )

        # Assistant 실행 상태 확인
        check_run_status(client, thread.id, run.id, calendar_service)

        thread_messages = client.beta.threads.messages.list(thread.id, order="desc", limit=1)
        
        # chat_history에 대화 내용 저장
        chat_history.append([None, user_input])
        if thread_messages.data:
            latest_message = thread_messages.data[0]
            if latest_message.role == "assistant":
                print(f"Assistant: {latest_message.content[0].text.value}")
                response = latest_message.content[0].text.value
                chat_history.append([response, None])
                return "", chat_history



# 초기화
def chat_reset(user_input, chat_history):
    thread = client.beta.threads.create()
    thread_id = thread.id
    user_input = ""
    chat_history = []
    return "", chat_history



In [105]:
answer = start_chat("안녕하세요", chat_history)
print(answer[0])

character is friendly_character
Assistant: 안녕하세요! 오늘 하루 잘 보내고 계시나요? 😊 



## Google Calendar API

In [111]:
# 구글 캘린더 연동


from datetime import datetime
import pytz
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import boto3

import os
from google.oauth2 import service_account

'''구글 캘린더 인증 부분 수정'''


# # Google Calendar API 자격 증명 설정

# 캘린더 인증 및 이벤트 변경을 위한 클래스
class CalendarService:
    def __init__(self, calendar_id='primary',timezone='Asia/Seoul'):
        # self.service_account_file = service_account_file
        self.SCOPES = ["https://www.googleapis.com/auth/calendar", "https://www.googleapis.com/auth/calendar.events"]
        self.calendar_id = calendar_id
        self.calendar_service = self.get_calendar_service()
        self.timezone = timezone
    # def get_calendar_service(self):
    #     """Shows basic usage of the Google Calendar API.
    #     Prints the start and name of the next 10 events on the user's calendar.
    #     """
    #     creds = None
    #     # The file token.json stores the user's access and refresh tokens, and is
    #     # created automatically when the authorization flow completes for the first time.
    #     try:
    #         if os.path.exists("token.json"):
    #             creds = Credentials.from_authorized_user_file("token.json", self.SCOPES)
    #         # If there are no (valid) credentials available, let the user log in.
    #         if not creds or not creds.valid:
    #             if creds and creds.expired and creds.refresh_token:
    #                 creds.refresh(Request())
    #             else:
    #                 flow = InstalledAppFlow.from_client_secrets_file(
    #                     "credentials.json", self.SCOPES
    #                 )
    #                 creds = flow.run_local_server(port=0)
    #             # Save the credentials for the next run
    #             with open("token.json", "w") as token:
    #                 token.write(creds.to_json())
    #         return build('calendar', 'v3', credentials=creds)
    #     except Exception as e:
    #         print(f"Error in getting calendar service: {e}")
    #         return None
        
    # def get_secret():

        # secret_name = "google_oauth_secret_key"
        # region_name = "us-west-1"

        # # Create a Secrets Manager client
        # session = boto3.session.Session()
        # client = session.client(
        #     service_name='secretsmanager',
        #     region_name=region_name
        # )

        # try:
        #     get_secret_value_response = client.get_secret_value(
        #         SecretId=secret_name
        #     )
        # except ClientError as e:
        #     # For a list of exceptions thrown, see
        #     # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        #     raise e

        # secret = get_secret_value_response['SecretString']

    def get_calendar_service(self):
        """Shows basic usage of the Google Calendar API.
        Prints the start and name of the next 10 events on the user's calendar.
        """
        creds = None
        # The file token.json stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first
        # time.
        if os.path.exists("token.json"):
            creds = Credentials.from_authorized_user_file("token.json", self.SCOPES)
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    "credentials.json", self.SCOPES
                )
                creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
            with open("token.json", "w") as token:
                token.write(creds.to_json())
        
        return build('calendar', 'v3', credentials=creds)
        
    #이벤트 생성
    def create_event(self, event):
        try:
            event_result = self.calendar_service.events().insert(calendarId=self.calendar_id, body=event).execute()
            return event_result
        except Exception as e:
            print(f"Error in creating event: {e}")
            return None

    #이벤트 가져오기
    def get_event(self, event_id):
        try:
            event = self.calendar_service.events().get(calendarId=self.calendar_id, eventId=event_id).execute()
            return event
        except Exception as e:
            print(f"Error in getting event: {e}")
            return None

    #이벤트 업데이트
    def update_event(self, event_id, updated_event):
        try:
            event = self.calendar_service.events().update(calendarId=self.calendar_id, eventId=event_id, body=updated_event).execute()
            return event
        except Exception as e:
            print(f"Error in updating event: {e}")
            return None

    #이벤트 삭제
    def delete_event(self, event_id):
        try:
            return self.calendar_service.events().delete(calendarId=self.calendar_id, eventId=event_id).execute()
        except Exception as e:
            print(f"Error in deleting event: {e}")
            return None

    def get_conflicts(self, start_time, end_time):
        try:
            events_result = self.calendar_service.events().list(
                calendarId=self.calendar_id,
                timeMin=start_time,
                timeMax=end_time,
                singleEvents=True,
                orderBy='startTime'
            ).execute()
            events = events_result.get('items', [])
            return events
        except Exception as e:
            print(f"Error in getting conflicts: {e}")
            return None

    def get_events(self, start_time, end_time):
        events_result = self.calendar_service.events().list(
            calendarId=self.calendar_id,
            timeMin=start_time,
            timeMax=end_time,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        events = events_result.get('items', [])
        return events

    def add_calendar_owner(self, user_email):
      calendar_id = 'primary'
      rule = {
          'scope': {
              'type': 'user',
              'value': user_email,
          },
          'role': 'owner'
      }
      created_rule = self.calendar_service.acl().insert(calendarId=calendar_id, body=rule).execute()
      return created_rule

    def get_current_datetime_and_timezone(self):

        timezone_str = self.timezone
        timezone = pytz.timezone(timezone_str)

        now = datetime.now(timezone)

        datetime_str = now.strftime('%Y-%m-%d %H:%M:%S %Z%z')

        return {
            "datetime": datetime_str,
            "timezone": timezone_str
        }



def execute_function(calendar_service, name, args_dict):

    method = getattr(calendar_service, name, None)

    if not method:
        raise ValueError(f"Method '{name}' not found in CalendarService class.")

    return method(**args_dict)

global calendar_service
calendar_service = CalendarService('primary')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=633821005299-ou61ditaviamu3rl6a9rkthn13nb4018.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A62746%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.events&state=PuJ4AIw9lvzhEJHUMPsp8SoDLgbphX&access_type=offline


# Web Deploy

In [112]:
# Gradio UI 설정_character 수정_재수정
# Chatbot 실행
import gradio as gr
with gr.Blocks() as app:
    gr.Markdown("## 마음일정")
    
    def chat_ui():
        with gr.Column():
            chat_history = gr.Chatbot(value=[[None, "안녕 나는 마음이야"]], show_label=False)
            with gr.Row():
                user_input = gr.Text(lines=1, placeholder="", container=False, scale=10)
                cb_send_btn = gr.Button(value="reply", size="small", scale=1)
                reset_btn = gr.Button(value="reset", size="small", scale=1)


                cb_send_btn.click(fn=start_chat, inputs=[user_input, chat_history], outputs=[user_input, chat_history])
                user_input.submit(fn=start_chat, inputs=[user_input, chat_history], outputs=[user_input, chat_history])
                reset_btn.click(fn=chat_reset, inputs=[user_input, chat_history], outputs=[user_input, chat_history])
            return chat_history, user_input, cb_send_btn, reset_btn
    with gr.Tab(character):
        character = "friendly_character"
        chat_ui()
        

app.launch(share=True)

Running on local URL:  http://127.0.0.1:7864
Running on public URL: https://67f9e7969fd83fc28e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


character is friendly_character
Assistant: 맞아, 가을이 왔지! 날씨도 선선해지고, 나뭇잎도 색이 변해서 예쁘더라. 가을에 하고 싶은 일이나 계획 있는지 궁금해. 😊🍂 근데, 일정 확인은 다시 부탁할게!
character is friendly_character
Assistant: 지금은 2024년 9월 24일 오후 5시 38분이야. 벌써 오후네! 이 시간에 뭘 하고 싶어? 😊
character is friendly_character
Assistant: 오늘은 특별한 일정이 없네! 자유 시간이라서 뭐든 할 수 있어. 가을을 만끽하면서 산책을 나가보는 게 어때? 아니면 좋아하는 책이나 영화도 좋을 것 같아. 😊 뭔가 계획을 세우고 싶으면 말해줘! 도와줄게.
character is friendly_character


# Test Part


In [11]:
os.environ['GOOGLE_CLIENT_ID']

'633821005299-ou61ditaviamu3rl6a9rkthn13nb4018.apps.googleusercontent.com'